### FFMPEG
#### https://pypi.org/project/ffmpeg-python/

In [ ]:
# Для установки библиотеки (ячейку достаточно выполнить один раз)
!pip install ffmpeg-python

In [ ]:
# Импорт библиотеки
import ffmpeg

In [ ]:
# Проверка деления
# Путь до видео
video_path = "/Users/vladislav/Documents/JupyterNotebook/ObjectDetection/Maintenance Hand Segmentation/Python Segmentation/pump_dataset/pump_videos/test_pump_video.mp4"
stream = ffmpeg.input(video_path)

stream = ffmpeg.trim(stream, start_frame=236, end_frame=241)
stream = ffmpeg.output(stream, "test2.mp4") # Отделенные кадры видео сохранятся в ту же папку, где расположен этот notebook
ffmpeg.run(stream)

In [ ]:
# Секунды, на которых происходит смена действия для видео test_pump_video.mp4
cuts_number = [6,16,27,40,52,70,80,100,120,130,164,172,178,184,225,300]

In [ ]:
# Алгоритм деления:
# start_frame - первое значение в массиве, end_frame - следующее значение в массиве
# Повторяем предыдущее действие до ех пор пока не дойдем до последнего элемента в массиве cuts_number
for i in range(len(cuts_number)):
    if i+1 != len(cuts_number):
        stream = ffmpeg.input("/Users/vladislav/Documents/JupyterNotebook/ObjectDetection/Maintenance Hand Segmentation/Python Segmentation/pump_dataset/pump_videos/test_pump_video.mp4")
        stream = ffmpeg.trim(stream, start_frame=cuts_number[i], end_frame=cuts_number[i+1])
        filename = "detection_" + str(cuts_number[i]) + "_" + str(cuts_number[i+1]) + ".mp4"
        stream = ffmpeg.output(stream, filename)
        ffmpeg.run(stream)
        print(cuts_number[i], " ", cuts_number[i+1])

### SceneDetectPy
#### Еще одна библиотека для деление видео на сцены https://pypi.org/project/scenedetect/

In [1]:
# Установка библиотеки
!pip install scenedetect

In [4]:
video_path = "/Users/vladislav/Documents/JupyterNotebook/ObjectDetection/Maintenance Hand Segmentation/Python Segmentation/pump_dataset/pump_videos/test_pump_video.mp4"

In [5]:
#To get started, there is a high level function in the library that performs content-aware scene detection on a video (try it from a Python prompt):
from scenedetect import detect, ContentDetector
scene_list = detect(video_path, ContentDetector())

In [6]:
scene_list

[(00:00:00.000 [frame=0, fps=24.999], 00:02:33.808 [frame=3845, fps=24.999]),
 (00:02:33.808 [frame=3845, fps=24.999],
  00:03:44.172 [frame=5604, fps=24.999]),
 (00:03:44.172 [frame=5604, fps=24.999],
  00:03:46.292 [frame=5657, fps=24.999]),
 (00:03:46.292 [frame=5657, fps=24.999],
  00:04:06.813 [frame=6170, fps=24.999]),
 (00:04:06.813 [frame=6170, fps=24.999],
  00:04:38.934 [frame=6973, fps=24.999]),
 (00:04:38.934 [frame=6973, fps=24.999],
  00:04:39.855 [frame=6996, fps=24.999]),
 (00:04:39.855 [frame=6996, fps=24.999],
  00:04:42.455 [frame=7061, fps=24.999]),
 (00:04:42.455 [frame=7061, fps=24.999],
  00:04:59.256 [frame=7481, fps=24.999]),
 (00:04:59.256 [frame=7481, fps=24.999],
  00:05:07.376 [frame=7684, fps=24.999])]

In [7]:
#Try calling print(scene_list), or iterating over each scene:
from scenedetect import detect, ContentDetector
scene_list = detect(video_path, ContentDetector())
for i, scene in enumerate(scene_list):
    print('    Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
        i+1,
        scene[0].get_timecode(), scene[0].get_frames(),
        scene[1].get_timecode(), scene[1].get_frames(),))

#We can also split the video into each scene if ffmpeg is installed (mkvmerge is also supported):
from scenedetect import detect, ContentDetector, split_video_ffmpeg
scene_list = detect(video_path, ContentDetector())
split_video_ffmpeg(video_path, scene_list)

#For more advanced usage, the API is highly configurable, and can easily integrate with any pipeline. This includes using different detection algorithms, splitting the input video, and much more. The following example shows how to implement a function similar to the above, but using the scenedetect API:
from scenedetect import open_video, SceneManager, split_video_ffmpeg
from scenedetect.detectors import ContentDetector
from scenedetect.video_splitter import split_video_ffmpeg

def split_video_into_scenes(video_path, threshold=27.0):
    # Open our video, create a scene manager, and add a detector.
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(
        ContentDetector(threshold=threshold))
    scene_manager.detect_scenes(video, show_progress=True)
    scene_list = scene_manager.get_scene_list()
    split_video_ffmpeg(video_path, scene_list, show_progress=True)

    Scene  1: Start 00:00:00.000 / Frame 0, End 00:02:33.808 / Frame 3845
    Scene  2: Start 00:02:33.808 / Frame 3845, End 00:03:44.172 / Frame 5604
    Scene  3: Start 00:03:44.172 / Frame 5604, End 00:03:46.292 / Frame 5657
    Scene  4: Start 00:03:46.292 / Frame 5657, End 00:04:06.813 / Frame 6170
    Scene  5: Start 00:04:06.813 / Frame 6170, End 00:04:38.934 / Frame 6973
    Scene  6: Start 00:04:38.934 / Frame 6973, End 00:04:39.855 / Frame 6996
    Scene  7: Start 00:04:39.855 / Frame 6996, End 00:04:42.455 / Frame 7061
    Scene  8: Start 00:04:42.455 / Frame 7061, End 00:04:59.256 / Frame 7481
    Scene  9: Start 00:04:59.256 / Frame 7481, End 00:05:07.376 / Frame 7684
